# Known Problem:
- **Unresolved**
    1. how to solve for eigen vectors
- **Partially Resolved**
    1. Certain websites cause the error of, specified here: https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests, which is a connection error. 
        - this appears to be windows specific as this was not a relevant issue when testing code in ubuntu 22.04, some forum reflect similar message
        - FAILED SOLUTION: first response in that stackoverflow page, still kept the stuff bc it seemed like better QOL
        - ATTEMPTED SOLUTION: used try and except. NOTE: this solution is not ideal as it does not resolve the issue, it just ignores it and moves on.
- **Resolved**
    1. The search "what is a taylor series" returns SSLCertVerification Error.
        - this resolution does not consider the security of the websites being accessed, potentially use try except to have the request verify when possible and not break when cannot verify
        - ATTEMPTED SOLUTION: used try except to use verify=False when ssl error occurs. Nested try except to ensure that both connection error and ssl error are handled.

    

    

# Part 1: Data Acquisition & Preparation

## Collecting & Preprocessing Our Data

In [1]:
#imports
import requests #send HTTP requests to web pages and retrieve their content
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from urllib.parse import urlparse, urljoin
import pandas as pd
from nltk.corpus import stopwords
import http.client
import re

In [2]:
import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
#uncomment if you cannot properly download punkt

nltk.download('punkt')
#ssl._create_default_https_context = ssl._create_default_https_context
#uncomment if you cannot properly download punkt

#remove filler words from an example column
nltk.download("stopwords")

nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/williamzhang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/williamzhang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/williamzhang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
#Working implementation

# Define the search query and retrieve search results
search_query = input("What would you like to search up?")
search_engine_url = "https://www.google.com/search?q="

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Number of search result pages to retrieve (each page typically has 10 results)
num_pages = 5

search_results = []

def verify_ssl_certificate(url):
    try:
        response = requests.get(url, verify=True, headers=headers)
        return True
    except requests.exceptions.SSLError:
        return False

for page in range(num_pages):
    # Calculate the start index for the current page
    start_index = page * 10

    try:
        # Make a GET request to retrieve the search result page
        response = requests.get(search_engine_url + search_query + f"&start={start_index}", headers=headers)
        response.raise_for_status()  # Raise an exception for non-2xx status codes

    except requests.exceptions.RequestException as e:
        print("Error occurred for page", page+1)
        print("Exception:", e)
        continue

    # Parse and extract information from the search result page
    soup = BeautifulSoup(response.content, "html.parser")

    for result in soup.select("div.g"):
        title_element = result.select_one("h3")
        if title_element:
            title = title_element.get_text()
            url_element = result.find("a")
            if url_element:
                url = url_element["href"]
                parsed_url = urlparse(url)
                if parsed_url.scheme == "":
                    # Relative URL, convert it to absolute URL
                    base_url = response.url
                    url = urljoin(base_url, url)

                # Verify SSL certificate before making the GET request
                if verify_ssl_certificate(url):
                    try:
                        print("Trying:", url)
                        result_response = requests.get(url, headers=headers)
                        result_response.raise_for_status()  # Raise an exception for non-2xx status codes
                        result_soup = BeautifulSoup(result_response.content, "html.parser")
                        text = result_soup.get_text()
                        text = ' '.join(text.split())
                        search_results.append({"Title": title, "URL": url, "Text": text})
                    except http.client.IncompleteRead:
                        print("Error: Incomplete read for:", url)
                        continue
                    except Exception as e:
                        print("Error: Failed to retrieve content for:", url)
                        print("Exception:", e)
                        continue

                # Make a GET request to the URL of the search result
                else:
                    print("SSL certificate verification failed for:", url)
                    continue
                    

print("Grabbed search results")

# Process the search results as desired

Trying: https://www.ibm.com/topics/artificial-intelligence
Trying: https://builtin.com/artificial-intelligence
Trying: https://www.techtarget.com/searchenterpriseai/definition/AI-Artificial-Intelligence
Trying: https://www.brookings.edu/articles/what-is-artificial-intelligence/
Trying: https://www.oracle.com/artificial-intelligence/what-is-ai/
Trying: https://www.britannica.com/technology/artificial-intelligence
Trying: https://www.mckinsey.com/featured-insights/mckinsey-explainers/what-is-ai
Trying: https://www.zdnet.com/article/what-is-ai-heres-everything-you-need-to-know-about-artificial-intelligence/
Trying: https://www.sas.com/en_us/insights/analytics/what-is-artificial-intelligence.html
Trying: https://en.wikipedia.org/wiki/Artificial_intelligence
Trying: https://www.investopedia.com/terms/a/artificial-intelligence-ai.asp
Trying: https://www.hpe.com/us/en/what-is/artificial-intelligence.html
Trying: https://hai.stanford.edu/sites/default/files/2020-09/AI-Definitions-HAI.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Trying: https://www.mygreatlearning.com/blog/what-is-artificial-intelligence/
Trying: https://www.state.gov/artificial-intelligence/
Trying: https://azure.microsoft.com/en-us/resources/cloud-computing-dictionary/artificial-intelligence-vs-machine-learning/
Trying: https://www.accenture.com/us-en/insights/artificial-intelligence-summary-index
Trying: https://www.youtube.com/watch?v=ad79nYk2keg
Trying: https://aws.amazon.com/machine-learning/what-is-ai/
Trying: https://www.simplilearn.com/tutorials/artificial-intelligence-tutorial/what-is-artificial-intelligence
Trying: https://azure.microsoft.com/en-us/resources/cloud-computing-dictionary/artificial-intelligence-vs-machine-learning/
Trying: https://aws.amazon.com/machine-learning/what-is-ai/
Trying: https://www.netapp.com/artificial-intelligence/what-is-artificial-intelligence/
Trying: https://www.nist.gov/artificial-intelligence
Trying: https://www.youtube.com/watch?v=ad79nYk2keg
Trying: https://www.gartner.com/en/topics/artificial-int

In [24]:
df = pd.DataFrame(search_results)
df_bart = pd.DataFrame(search_results)

# Extract domain names from URLs and set as the index for the dataframe
tld_pattern = re.compile(r'\.[a-zA-Z]+$')
df['Domain'] = df['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df.insert(0, 'Domain', df.pop('Domain'))

df_bart['Domain'] = df_bart['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df_bart.insert(0, 'Domain', df_bart.pop('Domain'))
#Handles the "Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER" exception
df_bart['Text'] = df_bart['Text'].str.replace('\ufffd', '')

#unfiltered dataframe
print(len(df))
df.head()

48


,Domain,Title,URL,Text
0,ibm,What is Artificial Intelligence (AI),https://www.ibm.com/topics/artificial-intellig...,What is Artificial Intelligence (AI) ? | IBM W...
1,builtin,Artificial Intelligence (AI): What Is AI and H...,https://builtin.com/artificial-intelligence,Artificial Intelligence (AI): What Is AI and H...
2,techtarget,What is artificial intelligence (AI)? - AI def...,https://www.techtarget.com/searchenterpriseai/...,What is artificial intelligence (AI)? - AI def...
3,brookings,What is artificial intelligence?,https://www.brookings.edu/articles/what-is-art...,What is artificial intelligence? | Brookings E...
4,oracle,What Is AI? Learn About Artificial Intelligence,https://www.oracle.com/artificial-intelligence...,What is Artificial Intelligence (AI)? | Oracle...


In [25]:
stop_words = stopwords.words("english") #english filler words

def preprocess(text):
    '''
    Input a text block and filter out unneeded characters
    returns a filtered text block in the form of a str
    Function filters whitespace, numbers, special characters, stopwords; handles case normalization 
    '''
    # Remove special characters and numbers
    cleaned_text = re.sub(r"[^a-zA-Z]+", " ", text)
    
    # Convert to lowercase and split into words
    words = cleaned_text.lower().split()
    
    # Remove stop words and single-character words
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1]
    lemmatized_text = " ".join([WordNetLemmatizer().lemmatize(word) for word in filtered_words])
    
    return lemmatized_text

def bart_preprocess(text):

    words = text.split()
    lemmatized_text = " ".join([WordNetLemmatizer().lemmatize(word) for word in words])

    return lemmatized_text


In [26]:
example = df["Text"][0]
processed_example = preprocess(example)

print("length of unfiltered text: ", len(example))
print("length of filtered text: ", len(processed_example))

length of unfiltered text:  14820
length of filtered text:  10955


In [27]:
#Properly using the apply() function onto a pandas column
#removes both filler words and special characters
df["Text"] = df["Text"].apply(preprocess)
df_bart["Text"] = df_bart["Text"].apply(bart_preprocess)


In [28]:
print(len(df["Text"][0]))

10955


In [29]:
#filtered text for first row entry
df["Text"][0]

'artificial intelligence ai ibm artificial intelligence ai artificial intelligence leverage computer machine mimic problem solving decision making capability human mind explore watsonx sign ai update artificial intelligence number definition artificial intelligence ai surfaced last decade john mccarthy offer following definition paper pdf kb link resides outside ibm science engineering making intelligent machine especially intelligent computer program related similar task using computer understand human intelligence ai confine method biologically observable however decade definition birth artificial intelligence conversation denoted alan turing seminal work computing machinery intelligence pdf kb link resides outside ibm published paper turing often referred father computer science asks following question machine think offer test famously known turing test human interrogator would try distinguish computer human text response test undergone much scrutiny since publish remains important 

# Part 2: EDA, Feature Engineering, and Model Development

### Future Optimizations

1. Ignore website navigation for websites like (wikipedia, stackoverflow, etc.)


### Tasks

1. Research into transformer, BART model, to better understand model set up and how to get highest performance
2. Experiment with BART input (lemmatize/not, capitalization, etc.) to find best performance 
3. Try to implement BART model with recursion as per the drawn diagram 
4. Research into paralellization of transformer models with HuggingFace 

### Notes

1. To actually make project work, we need to create a few metrics that we can train a DL model on. Maybe, linguistic complexity (formulaic), sentiment analysis, text length, etc.

In [30]:
import torch
from sklearn.model_selection import train_test_split

# Assuming you have a DataFrame called 'df' containing your data
# Split the data into train and test sets (80% train, 20% test)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Split the train data further into train and validation sets (80% train, 20% validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Print the sizes of each dataset
print("Train set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Test set size:", len(test_data))

Train set size: 30
Validation set size: 8
Test set size: 10


In [31]:
test_data.reset_index().drop(["index"], axis = 1)
val_data.reset_index().drop(["index"], axis = 1)
train_data.reset_index().drop(["index"], axis = 1)

,Domain,Title,URL,Text
0,spiceworks,What Is Artificial Intelligence (AI)? Definiti...,https://www.spiceworks.com/tech/artificial-int...,artificial intelligence ai definition type goa...
1,law.cornell,artificial intelligence (AI) | Wex - Law.Corne...,https://www.law.cornell.edu/wex/artificial_int...,artificial intelligence ai wex u law lii legal...
2,britannica,"Artificial intelligence (AI) | Definition, Exa...",https://www.britannica.com/technology/artifici...,artificial intelligence ai definition example ...
3,azure.microsoft,Artificial intelligence (AI) vs. machine learn...,https://azure.microsoft.com/en-us/resources/cl...,artificial intelligence v machine learning mic...
4,mitsloan.mit,"Machine learning, explained",https://mitsloan.mit.edu/ideas-made-to-matter/...,machine learning explained mit sloan skip main...
5,datarobot,artificial intelligence (AI) - Wiki,https://www.datarobot.com/wiki/artificial-inte...,artificial intelligence artificial intelligenc...
6,techcrunch,Age of AI: Everything you need to know about a...,https://techcrunch.com/2023/06/28/age-of-ai-ev...,age ai everything need know artificial intelli...
7,en.wikipedia,Artificial intelligence,https://en.wikipedia.org/wiki/Artificial_intel...,artificial intelligence wikipedia jump content...
8,azure.microsoft,Artificial intelligence (AI) vs. machine learn...,https://azure.microsoft.com/en-us/resources/cl...,artificial intelligence v machine learning mic...
9,sas,Artificial Intelligence (AI): What it is and w...,https://www.sas.com/en_us/insights/analytics/w...,artificial intelligence ai matter sa skip main...


In [32]:
#Using Bart
# !pip install transformers
# !pip install torch
from transformers import BartForConditionalGeneration, AutoTokenizer

In [33]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
# device = "cuda:0" if torch.cuda.is_available() else "cpu"


In [15]:
len(df_bart["Text"][0])

41231

In [20]:
inputs = tokenizer([df_bart["Text"][2]],max_length = 1024,return_tensors="pt")
# model = model.to(device)
# outputs = model(**inputs)


In [21]:
summary_ids = model.generate(inputs["input_ids"], num_beams=1, min_length=0, max_length=30)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'What is RGB Color? – Nix Sensor Ltd – NIX Sensor Ltd Skip to content BRAND NEW Nix Mini 3'

In [38]:
from transformers import LEDTokenizer, LEDForConditionalGeneration

tokenizer = LEDTokenizer.from_pretrained("hyesunyun/update-summarization-bart-large-longformer")
model = LEDForConditionalGeneration.from_pretrained("hyesunyun/update-summarization-bart-large-longformer")

inputs = tokenizer([df_bart["Text"][2]],max_length = 10240,return_tensors="pt")
predicted_summary_ids = model.generate(inputs["input_ids"], num_beams=1, min_length=0, max_length=30)
tokenizer.batch_decode(predicted_summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'There is a need for more research in this field. There is a lack of high-quality evidence to guide practice. There are'